# Data cleaning
We have our raw dataset, now we need to make it ready for analysis

### Import libraries

In [151]:
import pandas as pd
import json
import warnings

### Load raw dataset

In [152]:
raw = pd.read_csv("../00_raw/china_raw.csv", dtype={"cidade": str})
raw

,day,city,expense,price,payment_source,category
0,12-May,Pequim,Didi pro templo do céu (tava fechado),38.42,Carol,Transporte
1,12-May,Pequim,Didi pro shopping das Pérolas (Hungqiao Market),13.30,Carol,Transporte
2,12-May,Pequim,Colar e brincos Carol + brinco presente da Lara,170.00,Carol,Compras/Presentes
3,12-May,Pequim,Didi pra Qianmen,14.80,Carol,Transporte
4,12-May,Pequim,Almoço Qianmen,64.00,Carol,Alimentação
...,...,...,...,...,...,...
125,29-May,Lhasa,Didi para Bokar,NaN,Diva,Transporte
126,29-May,Lhasa,Almoço,33.00,Diva,Alimentação
127,29-May,Lhasa,Compras Bokar Supermarket,105.00,Carol,Compras/Presentes
128,29-May,Lhasa,Massagem no aeroporto,30.00,Diva,Compras/Presentes


### Add missing rows
When checking the AI generated dataset we noticed two issues:
<br>
1- Some values are null
<br>
2- Others are simply absent (from the dataset and the original source)
<br>
There's also a third issue:
<br>
3- The price is sometimes for two people (Carol and Diva) and sometimes for three people (Carol, Diva and Renata). It's reasonable to say we want to find how much more one spends travelling to and in China.
<br><br>
The first issue was solved with **.apply()**:

In [153]:
# Issue 1: some expenses have an NA on the "price" column
no_price = raw.loc[pd.isna(raw["price"])]
no_price

,day,city,expense,price,payment_source,category
51,17-May,Pequim,Metrô,NaN,Diva,Transporte
97,22-May,Pequim,Didi para Tiannanmen,NaN,Diva,Transporte
98,22-May,Pequim,Almoço no museu,NaN,Diva,Alimentação
99,22-May,Pequim,Comprinhas museu,NaN,Diva,Compras/Presentes
103,24-May,Lhasa,Didi pro aeroporto,NaN,Carol,Transporte
112,27-May,Shigatse,Jantar,NaN,Carol,Alimentação
125,29-May,Lhasa,Didi para Bokar,NaN,Diva,Transporte


In [154]:
# Solution: apply and lambda
raw.loc[raw["price"].isna(), "price"] = raw.loc[raw["price"].isna()].apply(
    lambda row: 42 if "Didi para Tiannanmen" in row["expense"] else pd.NA, axis=1)
raw.loc[raw["price"].isna(), "price"] = raw.loc[raw["price"].isna()].apply(
    lambda row: 8 if "Metrô" in row["expense"] else pd.NA, axis=1)
raw.loc[raw["price"].isna(), "price"] = raw.loc[raw["price"].isna()].apply(
    lambda row: 165 if "Comprinhas museu" in row["expense"] else pd.NA, axis=1)
raw.loc[raw["price"].isna(), "price"] = raw.loc[raw["price"].isna()].apply(
    lambda row: 26 if "Almoço no museu" in row["expense"] else pd.NA, axis=1)
raw.loc[raw["price"].isna(), "price"] = raw.loc[raw["price"].isna()].apply(
    lambda row: 107 if "Didi pro aeroporto" in row["expense"] else pd.NA, axis=1)
raw.loc[raw["price"].isna(), "price"] = raw.loc[raw["price"].isna()].apply(
    lambda row: 52 if "Jantar" in row["expense"] else pd.NA, axis=1)
raw.loc[raw["price"].isna(), "price"] = raw.loc[raw["price"].isna()].apply(
    lambda row: 14 if "Didi para Bokar" in row["expense"] else pd.NA, axis=1)
raw

/var/folders/mp/9bbmxhdd5ms420hm5z9_6f000000gn/T/ipykernel_2341/3125365558.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[<NA> 42 <NA> <NA> <NA> <NA> <NA>]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  raw.loc[raw["price"].isna(), "price"] = raw.loc[raw["price"].isna()].apply(


,day,city,expense,price,payment_source,category
0,12-May,Pequim,Didi pro templo do céu (tava fechado),38.42,Carol,Transporte
1,12-May,Pequim,Didi pro shopping das Pérolas (Hungqiao Market),13.3,Carol,Transporte
2,12-May,Pequim,Colar e brincos Carol + brinco presente da Lara,170.0,Carol,Compras/Presentes
3,12-May,Pequim,Didi pra Qianmen,14.8,Carol,Transporte
4,12-May,Pequim,Almoço Qianmen,64.0,Carol,Alimentação
...,...,...,...,...,...,...
125,29-May,Lhasa,Didi para Bokar,14,Diva,Transporte
126,29-May,Lhasa,Almoço,33.0,Diva,Alimentação
127,29-May,Lhasa,Compras Bokar Supermarket,105.0,Carol,Compras/Presentes
128,29-May,Lhasa,Massagem no aeroporto,30.0,Diva,Compras/Presentes


I solved the second issue (values that are relevant expenses but weren't on the original notes) by creating those rows myself and including other major expenses that weren't kept on the original file, such as airfares, train tickets, hotels and tour agency packages. These were paid for in advance, but I kept the day they were used to make the timeline better.
<br><br>
I saved that file as a json called **missing_data.json**, so now i can add the rows to the raw dataset, and get a more completed dataframe:

In [155]:
# Open the json with the data that was still missing:
missing_data = pd.read_json("../00_raw/missing_data.json", dtype={"cidade": str})
missing_data.head(10)

,day,city,expense,price,payment_source,category
0,13-May,Datong,Da Tong Weidu International Hotel,292.00,Diva,Hotel
1,17-May,Suzhou,HanTin Premium Hotel,656.00,Renata,Hotel
2,18-May,Xangai,Homeinn Hotel,970.00,Renata,Hotel
3,18-May,Guangzhou,SunYat Sen University Kaifeng Hotel,1382.00,Renata,Hotel
4,13-May,Datong,trem de Pequim para Datong,378.00,Renata,Transporte
5,15-May,Datong,trem de Datong para Pequim,366.00,Renata,Transporte
6,17-May,Suzhou,trem de Pequim para Suzhou,1224.00,Renata,Transporte
7,18-May,Suzhou,trem de Suzhou para Xangai,168.00,Renata,Transporte
8,20-May,Xangai,Avião de Xangai para Pequim,1460.00,Renata,Transporte
9,24-May,Lhasa,Avião ida e volta de Pequim para Lhasa,9169.11,Paula,Transporte


In [156]:
# Now let's concatenate both datasets
df_concat = pd.concat([raw, missing_data])
df_concat

,day,city,expense,price,payment_source,category
0,12-May,Pequim,Didi pro templo do céu (tava fechado),38.42,Carol,Transporte
1,12-May,Pequim,Didi pro shopping das Pérolas (Hungqiao Market),13.3,Carol,Transporte
2,12-May,Pequim,Colar e brincos Carol + brinco presente da Lara,170.0,Carol,Compras/Presentes
3,12-May,Pequim,Didi pra Qianmen,14.8,Carol,Transporte
4,12-May,Pequim,Almoço Qianmen,64.0,Carol,Alimentação
...,...,...,...,...,...,...
31,02-Jun,Guangzhou,Didi,18.79,Carol,Transporte
32,02-Jun,Pequim,Didi do aeroporto,110.0,Renata,Transporte
33,03-Jun,Pequim,Massagem nos pés,156.0,Carol,Compras/presentes
34,03-Jun,Pequim,Didi para o aeroporto,90.0,Renata,Transporte


In [157]:
# And reorder the rows by day
df_concat = df_concat.sort_values("day")
df_concat

,day,city,expense,price,payment_source,category
27,01-Jun,Guangzhou,Jantar,120.0,Diva,Alimentação
26,01-Jun,Guangzhou,Cruzeiro Rio das Pérolas,369.0,Renata,Ingressos
23,01-Jun,Guangzhou,Almoço,90.0,Diva,Alimentação
22,01-Jun,Guangzhou,Carro do aeroporto para o hotel,130.0,Renata,Transporte
32,02-Jun,Pequim,Didi do aeroporto,110.0,Renata,Transporte
...,...,...,...,...,...,...
18,30-May,Pequim,Lenços de seda,1145.0,Paula,Compras/presentes
17,30-May,Pequim,Almoço no bairro da Renata,1234.0,Diva,Alimentação
21,30-May,Pequim,Metrô,14.0,Paula,Transporte
16,30-May,Pequim,Supermercado,21.3,Carol,Compras/presentes


In [158]:
# Looks like our date format isn't helpful once there are two months involved.
# Let's fix it:
df_concat["day_cleaned"] = pd.Series(df_concat["day"])
df_concat["day_cleaned"] = pd.to_datetime(df_concat["day_cleaned"], format='%d-%b')
df_concat = df_concat.sort_values("day_cleaned", ascending=True)
df_concat["day_cleaned"] = df_concat["day_cleaned"].dt.strftime('%b-%d')
df_concat.drop("day", axis=1, inplace=True)

In [159]:
df = df_concat[["day_cleaned",
         "city",
         "expense",
         "price",
         "payment_source",
         "category"]]
df

,day_cleaned,city,expense,price,payment_source,category
14,May-11,Pequim,Táxi do aeroporto para a casa da Renata,87.0,Carol,Transporte
10,May-12,Pequim,Didi pro restaurante de dumpling fritos,49.93,Carol,Transporte
2,May-12,Pequim,Colar e brincos Carol + brinco presente da Lara,170.0,Carol,Compras/Presentes
8,May-12,Pequim,3 Baralhos,90.0,Carol,Compras/Presentes
7,May-12,Pequim,Mountain Coffee,25.0,Carol,Alimentação
...,...,...,...,...,...,...
31,Jun-02,Guangzhou,Didi,18.79,Carol,Transporte
32,Jun-02,Pequim,Didi do aeroporto,110.0,Renata,Transporte
33,Jun-03,Pequim,Massagem nos pés,156.0,Carol,Compras/presentes
35,Jun-03,Pequim,Taobao e Meituan,840.47,Renata,Compras/presentes


As for the third issue, I'm going to solve it using this logic:
- On the cities only my mother and I visited (Beijing, Datong, Shanghai and Shigatse), I'll divide the price of expenses by 2.
- On the other places (Suzhou, Lhasa and Guangzhou), I'll divide it by 3.

In [160]:
# Create a dataset only with the expenses in Beijing, Datong, Shanghai and Shigatse and divide the price by TWO:
trip_for_two = df[df['city'].isin(['Pequim', 'Datong', 'Shanghai', 'Shigatse'])]
trip_for_two["price"].astype(float)
trip_for_two["price_per_capita"] = trip_for_two["price"]/2
trip_for_two

/var/folders/mp/9bbmxhdd5ms420hm5z9_6f000000gn/T/ipykernel_2341/3479853598.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_for_two["price_per_capita"] = trip_for_two["price"]/2


,day_cleaned,city,expense,price,payment_source,category,price_per_capita
14,May-11,Pequim,Táxi do aeroporto para a casa da Renata,87.0,Carol,Transporte,43.5
10,May-12,Pequim,Didi pro restaurante de dumpling fritos,49.93,Carol,Transporte,24.965
2,May-12,Pequim,Colar e brincos Carol + brinco presente da Lara,170.0,Carol,Compras/Presentes,85.0
8,May-12,Pequim,3 Baralhos,90.0,Carol,Compras/Presentes,45.0
7,May-12,Pequim,Mountain Coffee,25.0,Carol,Alimentação,12.5
...,...,...,...,...,...,...,...
16,May-30,Pequim,Supermercado,21.3,Carol,Compras/presentes,10.65
32,Jun-02,Pequim,Didi do aeroporto,110.0,Renata,Transporte,55.0
33,Jun-03,Pequim,Massagem nos pés,156.0,Carol,Compras/presentes,78.0
35,Jun-03,Pequim,Taobao e Meituan,840.47,Renata,Compras/presentes,420.235


In [161]:
# Create a dataset only with the expenses in Suzhou, Lhasa and Guangzhou and divide the price by THREE:
trip_for_three = df[df['city'].isin(['Suzhou', 'Lhasa', 'Guangzhou'])]
trip_for_three["price"].astype(float)
trip_for_three["price_per_capita"] = trip_for_three["price"]/3
trip_for_three

/var/folders/mp/9bbmxhdd5ms420hm5z9_6f000000gn/T/ipykernel_2341/1481526534.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_for_three["price_per_capita"] = trip_for_three["price"]/3


,day_cleaned,city,expense,price,payment_source,category,price_per_capita
1,May-17,Suzhou,HanTin Premium Hotel,656.0,Renata,Hotel,218.666667
6,May-17,Suzhou,trem de Pequim para Suzhou,1224.0,Renata,Transporte,408.0
7,May-18,Suzhou,trem de Suzhou para Xangai,168.0,Renata,Transporte,56.0
3,May-18,Guangzhou,SunYat Sen University Kaifeng Hotel,1382.0,Renata,Hotel,460.666667
55,May-18,Suzhou,Metro,4.0,Diva,Transporte,1.333333
57,May-18,Suzhou,Entrada Jardim do Administrador Humilde,40.0,Diva,Ingressos,13.333333
58,May-18,Suzhou,Cartão-postal,10.0,Carol,Compras/Presentes,3.333333
59,May-18,Suzhou,Café espresso + leite,20.0,Diva,Alimentação,6.666667
56,May-18,Suzhou,Entrada Jardim do Administrador Humilde,80.0,Carol,Ingressos,26.666667
105,May-24,Lhasa,Show Princesa Wejcheng,840.0,Tica,Ingressos,280.0


In [162]:
# Concatenate the rows from the two datasets
df = pd.concat([trip_for_two, trip_for_three]).sort_values("day_cleaned")
df

,day_cleaned,city,expense,price,payment_source,category,price_per_capita
27,Jun-01,Guangzhou,Jantar,120.0,Diva,Alimentação,40.0
26,Jun-01,Guangzhou,Cruzeiro Rio das Pérolas,369.0,Renata,Ingressos,123.0
23,Jun-01,Guangzhou,Almoço,90.0,Diva,Alimentação,30.0
22,Jun-01,Guangzhou,Carro do aeroporto para o hotel,130.0,Renata,Transporte,43.333333
31,Jun-02,Guangzhou,Didi,18.79,Carol,Transporte,6.263333
...,...,...,...,...,...,...,...
20,May-30,Pequim,Supermercado,19.39,Carol,Alimentação,9.695
19,May-30,Pequim,Brincos e colar de pérola,1234.0,Diva,Compras/presentes,617.0
18,May-30,Pequim,Lenços de seda,1145.0,Paula,Compras/presentes,572.5
17,May-30,Pequim,Almoço no bairro da Renata,1234.0,Diva,Alimentação,617.0


Now, for the final part in getting the data tidy, I'm going to specified the payment type.
<br><br>
This is important because China, unlike the US and Brazil, has the so-called superapps, such as Alipay, used to call a Didi, take the metro, order at restaurants and pay for things in stores, or WeChat, more common in more remote locations when a business or vendor wouldn't accept Alipay.
<br><br>
I can do this because I know that all expenses that have "Paula" as payment_source were paid for using our credit cards connected to Paula's bank account in the US.
<br><br>
And all the other names mean that the payment was made using Alipay, WeChat or other app purchases, like Taobao and Meituan. The money comes from Renata's bank account in China and stays in a digital wallet inside the apps.
<br><br>
I want to see **how much we've spent in total from each payment type**, and the *average spent in each purchase* from these two groups.

In [163]:
# Create new column "payment_type" and filling it according to the values in column "payment_source"
df["payment_type"] = df["payment_source"].apply(lambda x: "credit card" if x == "Paula" else "apps")
df

,day_cleaned,city,expense,price,payment_source,category,price_per_capita,payment_type
27,Jun-01,Guangzhou,Jantar,120.0,Diva,Alimentação,40.0,apps
26,Jun-01,Guangzhou,Cruzeiro Rio das Pérolas,369.0,Renata,Ingressos,123.0,apps
23,Jun-01,Guangzhou,Almoço,90.0,Diva,Alimentação,30.0,apps
22,Jun-01,Guangzhou,Carro do aeroporto para o hotel,130.0,Renata,Transporte,43.333333,apps
31,Jun-02,Guangzhou,Didi,18.79,Carol,Transporte,6.263333,apps
...,...,...,...,...,...,...,...,...
20,May-30,Pequim,Supermercado,19.39,Carol,Alimentação,9.695,apps
19,May-30,Pequim,Brincos e colar de pérola,1234.0,Diva,Compras/presentes,617.0,apps
18,May-30,Pequim,Lenços de seda,1145.0,Paula,Compras/presentes,572.5,credit card
17,May-30,Pequim,Almoço no bairro da Renata,1234.0,Diva,Alimentação,617.0,apps


Just some final steps to get the dataframe as tidy as possible:

In [137]:
# Rename the column "day_cleaned" to "date" so it's tidier
df = df.rename(columns={ "day_cleaned": "date"} )
df["price"] = df["price"].astype(float)
df["price"] = df["price"].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 166 entries, 14 to 34
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            166 non-null    object 
 1   city            166 non-null    object 
 2   expense         166 non-null    object 
 3   price           166 non-null    float64
 4   payment_source  166 non-null    object 
 5   category        166 non-null    object 
 6   payment_type    166 non-null    object 
dtypes: float64(1), object(6)
memory usage: 10.4+ KB


Okay, now my dataframe is ready for the analyses I wanna do.

In [117]:
df.to_csv("../04_tidy_data/china_df.csv", index=False)